# HW

In [ ]:
import numpy as np
import pandas as pd

##### Загрузите датасет titanic.csv

In [ ]:
df_titanic = pd.read_csv('titanic.csv')

In [ ]:
df_titanic.head()

In [ ]:
df_titanic

In [ ]:
a = set()
for i in df_titanic.Embarked:
    a.add(i)
a

In [ ]:
df_titanic.shape

In [ ]:
df_titanic.info()

> Обратите внимание на разное количество значений переменных
Если это так - то у вас пропуски в данных. Такое бывает. 
Например в этом датасете не получилось выяснить всех подробностей у некоторых людей в виду утери данных, невозможности узнать или восстановить их

In [ ]:
df_titanic.describe()

> Давайте возьмем для начала только несколько признаков, позже - вы увеличите их количество и сделаете соответствующую обработку.

In [ ]:
columns = ['Sex', 'Age', 'Pclass', 'Survived', 'Cabin', 'Fare', 'Embarked']

In [ ]:
simple_train = df_titanic[columns].loc[:500]
simple_test = df_titanic[columns].loc[500:]
simple_test = simple_test.reset_index(drop = True)

> мы пока не умеем на уровне алгоритма работать с пропущенными значениями. 
Представьте вам дают 5-мерный вектор у которого заполнены только три компоненты и другой такой же вектор но у него заполнены лишь две  компоненты.  Как померить их сходство????

distance_between( [1,0,1, NaN, NaN], [NaN, 1, 1, NaN, NaN] )


> один из вариантов - заполнить какими то значениями - например если это возраст - то заполнить средним по датасету или нулем.

>> на будущее - в sklearn есть такой модуль - http://scikit-learn.org/stable/modules/impute.html но сейчас можете заполнить пропуски средствами pandas - метод fillna()

##### Заполните пропущенные значения чем нибудь

In [ ]:
simple_train.Age = simple_train.Age.fillna(np.mean(simple_train.Age))
simple_test.Age = simple_test.Age.fillna(np.mean(simple_test.Age))

# можно inplace делать такое, а можно переприсваивать.

In [ ]:
simple_train.Cabin = simple_train.Cabin.fillna('H')
simple_test.Cabin = simple_test.Cabin.fillna('H')

In [ ]:
simple_train.Embarked = simple_train.Embarked.fillna('A')
simple_test.Embarked = simple_test.Embarked.fillna('A')

In [ ]:
simple_train

> мы чемпионы. Но мы понимаем, что алгоритм у нас метрический - он не понимает как померить близость векторов в которых содержатся строки

distance( [1,1,0,'мужик'], [1, 0, 1, 'принцесска'] )

> поэтому давайте заведем признак is_man который будет принимать значение 0 или 1.

##### Заведите бинарный признак отвечающий за пол. Строковый удалите

хинт - есть метод apply()

In [ ]:
simple_train.Embarked = simple_train.Embarked.apply(lambda x: ord(x))
simple_test.Embarked = simple_test.Embarked.apply(lambda x: ord(x))
simple_test

In [ ]:
simple_train.Cabin = simple_train.Cabin.apply(lambda x: ord(x[0]))
simple_test.Cabin = simple_test.Cabin.apply(lambda x: ord(x[0]))
simple_test

In [ ]:
simple_train.Sex = simple_train.Sex.apply(lambda x: 1 if x == 'male' else 0)
simple_test.Sex = simple_test.Sex.apply(lambda x: 1 if x == 'male' else 0)
simple_test

In [ ]:
max_ = simple_train.max(axis=0)
min_ = simple_train.min(axis=0)
simple_train = (simple_train - min_) / (max_ - min_)

max_ = simple_test.max(axis=0)
min_ = simple_test.min(axis=0)
simple_test = (simple_test - min_) / (max_ - min_)

In [ ]:
simple_train

Сформируйте трейн и тест выборку, где в X лежат признаки 'Sex', 'Age', 'Pclass', а в у - Survived

Результат - numpy array

hint - у pandas есть метод drop() и атрибут values; Пощупайте их

In [ ]:
X_train = simple_train.drop(columns='Survived').values
X_test = simple_test.drop(columns='Survived').values
y_train = simple_train['Survived'].values
y_test = simple_test['Survived'].values

In [ ]:
assert isinstance(X_train, np.ndarray)
assert isinstance(X_test, np.ndarray)
assert isinstance(y_train, np.ndarray)
assert isinstance(y_test, np.ndarray)

In [ ]:
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

In [ ]:
assert not np.any(np.isnan(X_train)), 'у вас в данных есть пропущенные значения. Заполните их или удалите'
assert not np.any(np.isnan(X_test)), 'у вас в данных есть пропущенные значения. Заполните их или удалите'

##### Реализуйте KNN. Сами, как делали это на лекции. 

Можете улучшить его.
(например использовать kd-tree для быстрого поиска ближайших соседей, 
 или обходить соседей и класть расстояния в мин-кучу - тогда вы быстрее будете набирать их, без сортировки.)

In [ ]:
class KNN:
    """
    simple KNN classifier with O(n) complexity for memory,
    and O(n log n) complexity - for nn search.
    """
    
    def __init__(self, n):
        self.n = n
        
    def fit(self, X, y):
        self.X = X
        self.y = y
    
    def predict(self, X):
        y_hat = []
        assert len(X.shape) == 2
        for point in X:
            dist = 0.
            for h, w in enumerate(point):
                dist += (self.X[:, h] - w)**2
            dist = np.sqrt(dist)
            
            idx = np.argsort(dist)[:self.n]
            
            counts = np.bincount(self.y[idx].astype(int))
            
            prediction = np.argmax(counts)
            y_hat.append(prediction)
            
        return np.array(y_hat)

In [ ]:
knn = KNN(22)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

##### посчитайте руками (не используя sklearn) качество вашего предсказания

In [ ]:
acc = np.mean(y_pred == y_test)
print('доля правильных ответов:', acc)

In [ ]:
np.unique(y_pred)

> Мы получили бейслайн решение. Оно должно давать скорр в районе 0.76

____

In [ ]:
### можете проверить себя

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

np.unique(y_pred)

print(accuracy_score(y_test, y_pred))

##### TODO продолжение домашки

In [ ]:
test = np.zeros(30)
for i in range(1,31):
    knn = KNN(i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = np.mean(y_pred == y_test)
    np.put(test,i-1,acc)
    print('{: f} при  n = {: d}'.format(test[i-1],i))
index  = np.argmax(test)
print('Лучший скорр при n = {: d} и он равен {: f}'.format(index+1,test[index]))